# Azimuthal Integral Tutorial

## Introduction

This tutorial demonstrates how to acquire an azimuthal integral profile from a multidimensional data set in pyXem. 

The data set is a 10x10x256x256 data set of a polycrystalline gold film acquired using a Medipix3 256 by 256 pixel detector. 

This functionality has been checked to run in pyxem-0.10.0 (November 2019). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/pyxem-demos/issues

# Contents

1. <a href='#loa'> Loading & Inspection</a>
2. <a href='#det'> Creating a Detector Object</a>
3. <a href='#cal'> Calculation Calibration Parameters</a>
4. <a href='#ai'> Performing Azimuthal Integration</a>

Import pyxem and other required libraries

In [ ]:
%matplotlib tk
import pyxem as pxm
import numpy as np

# <a id='loa'></a> 1. Loading and Inspection

Load polycrystalline SED data

In [ ]:
dp = pxm.load_hspy('ai-demo-data.hspy',
                  assign_to='electron_diffraction2d')

Check the data size and type

In [ ]:
dp

Apply an affine distortion correction

In [ ]:
dp.apply_affine_transformation(
    np.array([[0.99978285, 0.00341758, 0.],
             [0.00341758, 0.94621262, 0.],
             [0., 0., 1.]]),keep_dtype=True)

In [ ]:
calib = 0.009197
dp.set_diffraction_calibration(calib)
dp.set_scan_calibration(5)

Plot the data for inspection

In [ ]:
dp.plot(vmax=1000)

# <a id='det'></a> 2. Creating a Detector Object

To get an azimuthal integral, it is important to characterise the detector, particularly in case it is not well represented by a flat-field approximation. To do this, we use the PyFAI Detector class. For example, here we create a Medipix256x256 Detector object from pyXem. 

In [ ]:
from pyxem.detectors import Medipix256x256Detector

In [ ]:
detector = Medipix256x256Detector()

In [ ]:
detector

### 2.1. Generic Detector Object

pyXem also defines a generic detector object which can be used if you're uncertain about certain detector parameters, or need a temporary fix for something.

In [ ]:
from pyxem.detectors import GenericFlatDetector

In [ ]:
detector2 = GenericFlatDetector(256,256)

The 256, 256 refers to the size of the detector in its two dimensions.

# <a id='cal'></a> 3. Calculating Calibration Parameters

In addition to specifying the detector, to accurately calculate the curvature of the Ewald Sphere, it is important to specify a calibration. In addition, the wavelength is specified to do that calculation.

The calibration is calculated by knowing the camera length. Alternatively, by assuming a no curvature in the detector, it is possible to calculate the camera length from an "inverse angstroms per pixel" calibration value. We suggest calibrating to a gold pattern for a calibration value and using the latter (for electron microscopy).

In [ ]:
wavelength = 2.5079e-12 # in metres for 200 kV

### 3.1 By knowing the camera length accurately

In [ ]:
camera_length = 0.24 #in metres

### 3.2 By calculating the camera length from a calibration value

In [ ]:
pix_size = 55e-6 #change to 1 if using the GenericFlatDetector()
camera_length = pix_size / (wavelength * calib * 1e10)
print('Camera Length:', camera_length)

# <a id='ai'></a> 4. Performing Azimuthal Integration

Specify the origin (in pixels) as a list [x y]. If the origin moves, instead an array of origins can be passed instead.

In [ ]:
origin = [127.5, 127.5]

Get the azimuthal integral selecting 181 effective pixels in 1D

In [ ]:
ai = dp.get_azimuthal_integral(origin=origin,
                               detector_distance=camera_length,
                               detector=detector, 
                               wavelength=wavelength,
                               size_1d=181)

Inspect the integrated data. Plotted after cropping direct beam.

In [ ]:
ai.isig[0.2:].plot()